In [1]:
# Parameters
country = "Argentina"


In [2]:
# Parámetros del notebook - serán sobrescritos por Papermill
# Este es el valor por defecto que se usará si no se pasa el parámetro
country = "Argentina"

In [3]:
# Configuración y validación de parámetros
from pathlib import Path

print(f"Procesando país: {country}")

# Validar que se pasó un parámetro válido
if country == "Argentina":
    print("Advertencia: Usando país por defecto. Asegúrate de pasar el parámetro 'country' con Papermill")

# Configuración de directorios
data_dir = Path('data')
outdir = Path('output')
outdir.mkdir(parents=True, exist_ok=True)

# Crear nombre seguro para archivos
country_safe = country.replace('/', '_').replace(' ', '_')
print(f"Archivo de datos esperado: {country_safe}.csv")

Procesando país: Argentina
Advertencia: Usando país por defecto. Asegúrate de pasar el parámetro 'country' con Papermill
Archivo de datos esperado: Argentina.csv


In [4]:
# Importar librerías necesarias
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")


Librerías importadas correctamente


In [5]:
# Cargar y validar datos
fpath = data_dir / f'{country_safe}.csv'

if not fpath.exists():
    raise FileNotFoundError(f'Archivo de datos no encontrado: {fpath}')

print(f"Cargando datos desde: {fpath}")
df = pd.read_csv(fpath, parse_dates=['date'])

if 'confirmed' not in df.columns:
    raise KeyError('El CSV no contiene la columna "confirmed"')

print(f"Datos cargados: {len(df)} registros desde {df['date'].min()} hasta {df['date'].max()}")


Cargando datos desde: data\Argentina.csv
Datos cargados: 1143 registros desde 2020-01-22 00:00:00 hasta 2023-03-09 00:00:00


In [6]:
# Procesar y transformar datos
print("Aplicando transformaciones...")

# Ordenar por fecha
df = df.sort_values('date').reset_index(drop=True)

# Calcular nuevos casos diarios
df['new_confirmed'] = df['confirmed'].diff().fillna(df['confirmed'])
df['new_confirmed'] = df['new_confirmed'].clip(lower=0)  # No permitir valores negativos

# Calcular media móvil de 7 días
df['ma7_new'] = df['new_confirmed'].rolling(window=7, min_periods=1).mean()

print(f"Transformaciones aplicadas:")
print(f"   - Nuevos casos diarios calculados")
print(f"   - Media móvil de 7 días calculada")
print(f"   - Total de casos confirmados: {int(df['confirmed'].iloc[-1]):,}")
print(f"   - Últimos nuevos casos: {int(df['new_confirmed'].iloc[-1]):,}")


Aplicando transformaciones...
Transformaciones aplicadas:
   - Nuevos casos diarios calculados
   - Media móvil de 7 días calculada
   - Total de casos confirmados: 10,044,957
   - Últimos nuevos casos: 0


In [7]:
# Generar resumen estadístico
print("Generando resumen estadístico...")

summary = {
    'country': country,
    'total_confirmed': int(df['confirmed'].iloc[-1]) if len(df) else 0,
    'latest_new': int(df['new_confirmed'].iloc[-1]) if len(df) else 0,
    'n_days': int(len(df)),
    'first_date': str(df['date'].min()) if len(df) else None,
    'last_date': str(df['date'].max()) if len(df) else None,
    'max_daily': int(df['new_confirmed'].max()) if len(df) else 0,
    'avg_daily': round(float(df['new_confirmed'].mean()), 2) if len(df) else 0
}

# Guardar resumen en JSON
summary_path = outdir / f'{country_safe}_summary.json'
summary_path.write_text(json.dumps(summary, ensure_ascii=False, indent=2))
print(f"Resumen guardado en: {summary_path}")


Generando resumen estadístico...
Resumen guardado en: output\Argentina_summary.json


In [8]:
# Generar visualizaciones
if len(df) > 0:
    print("Generando visualizaciones...")
    
    # Configurar estilo de matplotlib
    import matplotlib
    plt.style.use('default')
    matplotlib.rcParams['figure.facecolor'] = 'white'
    matplotlib.rcParams['axes.facecolor'] = 'white'
    
    # Gráfico 1: Casos acumulados
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df['date'], df['confirmed'], color='#2E86AB', linewidth=2.5, label='Casos acumulados')
    ax.set_title(f'Casos confirmados acumulados - {country}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Casos confirmados', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Formatear eje Y con separadores de miles
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
    
    plt.tight_layout()
    cum_path = outdir / f'{country_safe}_cumulative.png'
    fig.savefig(cum_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Gráfico acumulado guardado: {cum_path}")
    
    # Gráfico 2: Nuevos casos diarios y media móvil
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(df['date'], df['new_confirmed'], color='#F24236', alpha=0.6, label='Nuevos casos diarios', width=1)
    ax.plot(df['date'], df['ma7_new'], color='#F6AE2D', linewidth=3, label='Media móvil 7 días')
    ax.set_title(f'Nuevos casos diarios y media móvil - {country}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Nuevos casos', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Formatear eje Y
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
    
    plt.tight_layout()
    daily_path = outdir / f'{country_safe}_daily_ma7.png'
    fig.savefig(daily_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Gráfico diario guardado: {daily_path}")
    
else:
    print("No hay datos para generar visualizaciones")


Generando visualizaciones...


Gráfico acumulado guardado: output\Argentina_cumulative.png


Gráfico diario guardado: output\Argentina_daily_ma7.png


In [9]:
# Resumen final del procesamiento
print("\n" + "="*50)
print(f"PROCESAMIENTO COMPLETADO PARA {country.upper()}")
print("="*50)
print(f"Total de casos confirmados: {summary['total_confirmed']:,}")
print(f"Últimos nuevos casos: {summary['latest_new']:,}")
print(f"Período: {summary['first_date']} a {summary['last_date']}")
print(f"Días de datos: {summary['n_days']}")
print(f"Máximo diario: {summary['max_daily']:,}")
print(f"Promedio diario: {summary['avg_daily']:.1f}")
print("="*50)



PROCESAMIENTO COMPLETADO PARA ARGENTINA
Total de casos confirmados: 10,044,957
Últimos nuevos casos: 0
Período: 2020-01-22 00:00:00 a 2023-03-09 00:00:00
Días de datos: 1143
Máximo diario: 139,853
Promedio diario: 8788.2
